In [1]:
import pandas as pd
import sklearn
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
import pickle

In [3]:
jan=pd.read_parquet("yellow_tripdata_2023-01.parquet")

In [4]:
jan.shape

(3066766, 19)

In [5]:
jan.tpep_pickup_datetime= pd.to_datetime(jan.tpep_pickup_datetime)
jan.tpep_dropoff_datetime= pd.to_datetime(jan.tpep_dropoff_datetime)
jan["duration"]=jan.tpep_dropoff_datetime-jan.tpep_pickup_datetime
jan.duration=jan.duration.apply(lambda td : td.total_seconds()/60)

In [6]:
jan.duration.std()

42.59435124195458

In [7]:
jan2=jan[(jan.duration >= 1) & (jan.duration <= 60)]

In [8]:
len(jan2)/len(jan)

0.9812202822125979

In [12]:
categorical= ["PULocationID", "DOLocationID"]
jan[categorical]=jan[categorical].astype("string")
dv= DictVectorizer()
numerical=["trip_distance"]
target="duration"

train_dict= jan[categorical].to_dict(orient="records")
X_train= dv.fit_transform(train_dict)

In [13]:
X_train.shape

(3066766, 518)

In [14]:
y_train=jan[target].values

lr=LinearRegression()
lr.fit(X_train, y_train)
y_pred= lr.predict(X_train)
mean_squared_error(y_train, y_pred, squared=False)

41.99649768735752

In [10]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error

In [ ]:
def readdata(filename):
    jan=pd.read_parquet(filename)
    jan.tpep_pickup_datetime= pd.to_datetime(jan.tpep_pickup_datetime)
    jan.tpep_dropoff_datetime= pd.to_datetime(jan.tpep_dropoff_datetime)
    jan["duration"]=jan.tpep_dropoff_datetime-jan.tpep_pickup_datetime
    jan.duration=jan.duration.apply(lambda td : td.total_seconds()/60)
    jan=jan[(jan.duration >= 1) & (jan.duration <= 60)]
    categorical= ["PULocationID", "DOLocationID"]
    jan[categorical]=jan[categorical].astype("string")
    return jan

In [ ]:
df_train=readdata("yellow_tripdata_2023-01.parquet")
df_val=readdata("yellow_tripdata_2023-02.parquet")

In [ ]:
df_train["PU_DU"]=df_train["PULocationID"]+ "_" + df_train["DOLocationID"]
df_val["PU_DU"]=df_val["PULocationID"]+ "_" + df_val["DOLocationID"]

In [ ]:
categorical= ["PU_DU"]
numerical=["trip_distance"]
target="duration"
dv= DictVectorizer()
train_dict= df_train[categorical + numerical].to_dict(orient="records")
X_train= dv.fit_transform(train_dict)

val_dict= df_val[categorical + numerical].to_dict(orient="records")
X_val= dv.transform(val_dict)

In [ ]:
y_train=df_train[target].values
y_val=df_val[target].values
lr=LinearRegression()
lr.fit(X_train, y_train)
y_pred= lr.predict(X_val)
mean_squared_error(y_val, y_pred, squared=False)

In [ ]:
with open("models/linear_regression.bin", "wb") as f_out:
    pickle.dump((dv, lr), f_out)